<img src="https://thevalley.es/wp-content/uploads/2016/11/thevalley-logo-negro.png" width="400"></img>


# Ejercicio avanzado sobre modelos de k-vecinos

Notebook por 
[David Rey Blanco](https://www.linkedin.com/in/davidreyblanco).

*Recursos*: Los códigos de esta sesión y recursos se pueden encontra en el [repositorio](https://github.com/davidreyblanco/ml-training) 
<br/>
<hr/>

In [3]:
# Importamos scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Importamos  train_test_split function
from sklearn.model_selection import train_test_split
# Tranformación de variables
from sklearn import preprocessing
import pandas as pd
import numpy as np
# Ignoramos los warning que no quedan muy bien
import warnings
warnings.filterwarnings('ignore')

# Carga de datos y descriptivos

El dataset que usaremos para el ejercicio corresponde a pisos para varias zonas de Madrid (nota: se ha generado en base a un modelo de valoración creado con información de portales inmobiliarios), los campos principales son:

* CODE codigo único del anuncio
* PRICE precio en euros
* AREA Superficie construida
* ROOMNUMBER Número de estancias (huecos)
* BATHNUMBER Número de baños
* STUDIO - Variable binaria  1 - Es un estudio 0 - No es un estudio
* ISPENTHHOUSE - Variable binaria  1 - Es un ático 0 - No es un ático
* DUPLEX - Variable binaria  1 - Es un dúplex 0 - No es un dúplex
* SWIMMINGPOOL - Variable binaria  1 - Tiene piscina - 0 No tiene piscina
* ELEVATOR - Variable binaria  1 - Tiene ascensor - 0 No tiene ascensor
* X - Coordenada geográfica X de la localización (longitud en EPGS:25830)
* Y - Coordenada geográfica X de la localización (latitud en EPGS:25830)

**Nota**: la proyección de las coordenadas es  la EPGS:25830 https://spatialreference.org/ref/epsg/etrs89-utm-zone-30n/ una unidad en esta medida representa un metro

## Descarga de datos y descriptivos básicos

Descargamos los datos desde el repositorio y extraemos algunos descriptivos

In [2]:
df_flats = pd.read_csv('https://raw.githubusercontent.com/davidreyblanco/ml-training/master/data/sample-flats-madrid.csv.gz', delimiter=';')
datos

,CODE,PRICE,AREA,ROOMNUMBER,BATHNUMBER,STUDIO,ISPENTHOUSE,DUPLEX,SWIMMINGPOOL,ELEVATOR,X,Y
count,2.733000e+03,2.733000e+03,2733.000000,2733.000000,2733.000000,2733.000000,2733.000000,2733.000000,2733.000000,2733.000000,2733.000000,2.733000e+03
mean,7.033958e+18,8.102975e+05,146.481888,2.935602,2.079034,0.019027,0.071716,0.027442,0.108306,0.903037,442558.737161,4.477613e+06
std,2.932817e+18,8.343778e+05,110.170071,1.478329,1.129423,0.136644,0.258064,0.163398,0.310823,0.295962,614.028907,6.612871e+02
min,6.034416e+15,4.828100e+04,22.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,441372.224765,4.476512e+06
25%,4.890341e+18,3.400020e+05,77.000000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,442120.762177,4.477055e+06
50%,9.223372e+18,5.252540e+05,108.000000,3.000000,2.000000,0.000000,0.000000,0.000000,0.000000,1.000000,442622.122753,4.477562e+06
75%,9.223372e+18,9.548910e+05,184.000000,4.000000,3.000000,0.000000,0.000000,0.000000,0.000000,1.000000,442998.347628,4.478111e+06
max,9.223372e+18,1.460009e+07,1170.000000,15.000000,10.000000,1.000000,1.000000,1.000000,1.000000,1.000000,443872.278545,4.478966e+06


## Construcción de nuevas características

Generamos la variable de euros metro cuadrado y la llamaremos **UNITPRICE**, vamos a revisar que tenemos:

In [10]:
df_flats['UNITPRICE'] = df_flats['PRICE'] / df_flats['AREA']
df_flats['UNITPRICE'].describe()

count      2733.000000
mean       5388.578865
std        4544.193344
min         629.748503
25%        4089.204255
50%        4987.116279
75%        6039.840000
max      211595.449275
Name: UNITPRICE, dtype: float64

## Descriptivos

Completamos nuestro análisis exploratorio

# Objetivos
Queremos construir un modelo de regresión basado en K-Vecinos que estime el precio en euros de un inmueble, para ello usarmos un algortimo que modele como target la variable *UNITPRICE* y nuestro resultado será el precio €/m² que nos de nuestro modelo multiplicado por la superficie del inmueble:

$$
Precio final = UNITPRICE_{KNN} \cdot Superficie
$$

# Tratamiento previo de variables

## Transformación de variables
¿Qué hacemos con las variables?, ¿las dejamos como están?, vemos que las escalas son muy diferentes, después tenemos variables booleanas (por suerte no tenemos variables categóricas) ...

## Bonus track en transformación de variables

¿Qué diferencias observamos cuando aplicamos distintos tipos de transformación sobre las variables continuas?:

* preprocessing.MinMaxScaler
* preprocessing.Normalizer
* preprocessing.PowerTransformer

[Transformers scikit-learn](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing)

## Selección de características
Este es quizás el paso más complicado de todos, porque requiere a veces la toma de decisiones en base a criterios expertos, ¿Qué características usamos?, ¿Cuáles será nuestra variable objetivo (o a predecir)?

# ¿Cómo funciona de bien nuestro modelo?

Para evaluar nuestro modelo debemos seleccionar una métrica de error, penalización, pérdida o como la queramos llamar. Debe representar lo mal que funciona nuestro modelo. Como es lógico debemos aspirar a un modelo que ofrezca el menor valor posible para esta función.

En los modelos de regresión las métricas más habituales son:

* MAE - Error medio en valor absoluto
* MEDAE - Error mediano en valor absoluto
* RMSE - Error cuadrático medio 

**Importante**: Los errores cuadráticos son interesantes porque nos ayudan a penalizar aquellas predicciones que son más extremas, en este caso limitan la dispersión de los errores producidos por el modelo

# Optimización de nuestro modelo

Hemo elegido un K=6 un poco de forma arbitraria (y porque es la recomendación del Banco de España en procesos de tasación), sin embargo algo nos dice que quizá no sea el K más adecuado, vamos a realizar una búsqueda

## Bonus track el K variable
Vamos a probar que pasa si con el K para distintos segmentos de nuestra muestra, por ejemplo en base a dos segmentos:

* Si tiene ascensor o no
* Si es estudio o no

¿Qué pasa con la estabilidad de nuestro K cuando nuestra muestra es pequeña (es estudio = Si)?

# Efecto de la medida de distancia
En principio podemos haber elegido alguna de las medidas estándar basadas en Minkowski (euclídea, manhattan, etc), aunque en general este es el segunddo gran reto de este tipo de modelos ¿qué medida de distancia tomo?

Podemos probar a evaluar el comportamiento (error) del modelo para distintas medidas de distancia, ¿qué vemos?

## Medida de distancia a medida
La medida de distancia es complicada vamos a construir una a medida que sea una distancia propia calculada como:

$$
 Distancia (piso A, piso B) = 0.8 \cdot Distancia_{Manhattan} [coordenadas(a,b)] + 0.2 \cdot Distancia_{Euclidea} [resto(a,b)]
$$


# Conclusiones
Ahora llega casi lo más complicado, ¿qué conclusiones hemos sacado de este ejercicio?, deberíamos poder decir algo sobre:

* ¿Qué ventajas inconvenientes vemos en este tipo de algoritmo versus modelso de árboles, regresión, etc?
* Política de tratamiento previo de los datos
* Efecto de las medidas de distancia utilizadas
* Como estimar el K Óptimo
* Sensibilidad de nuestro modelo ante falta de datos

# Referencias
[K-Neighbors Scikitlearn](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.neighbors)